<h1> BBHs merging catalog generator </h1> 

In the following, we'll implement a notebook that, given a certain volume of sky, will return a catalog of possible BBHs merging events.
The probability distribution implemented for the variables of the events, will be taken from [B. P. Abbott](https://arxiv.org/abs/1811.12940).
First of all, we need to import some modules ! 

In [ ]:
import numpy as np
import scipy.special as sc
import statistics as st
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

Let's start by defining the probability distribution in function of the masses, we'll assume that the regime $m_{min} \leq m_{2} \leq m_{1} \leq m_{max}$ holds :

In [ ]:
#Beware to alpha = 1. or beta_q = -1, the C_norm wouldn't work in that case

def MassDistr(m1, m2, m_min, m_max, alpha, beta_q):
    if(m1 >= m_min and m1 <= m_max and m2 <= m1 and m2 >= m_min) :
        q = m2/m1
        PhaseSpace = ((np.power(m_max, 1. - alpha) - np.power(m_min, 1. - alpha))/(1. - alpha))*(1./(beta_q - 1.))
        C_norm = 1./PhaseSpace
        return (C_norm*np.power(m1, (-alpha))*np.power(q, beta_q))
    else :
        return 0.

Then, we'll need a function that allow us to convert from redshift to Gigaparsec :

In [ ]:
def Z_to_Gpc(z):
    return ((z*c*(10**(-3)))/(H_0)) # only valid for z < 1

and the function that estimates the differential comoving volume in function of the redshift :

In [ ]:
#In the following function, the differential comoving volume in function of the redshift will be estimated as a spherical surface, it need to be integrated over dr to obtain the real volume 

def DeVC(z):
    r = Z_to_Gpc(z)    
    return (4.*np.pi*(r**2.))

Now let's define the auxiliary functions for the spin distribution density :

In [ ]:
def BetaSpinParameters(Expected_a, Var_a):
    expec_rel = (Expected_a/(1. - Expected_a))
    beta_a = ((expec_rel - Var_a*np.power(1. + expec_rel, 2.))/(Var_a*np.power(1. + expec_rel, 3.)))
    alpha_a = expec_rel*beta_a
    return alpha_a, beta_a

and hence, the spin distribution module density is given by :

In [ ]:
# Estimating the beta function use the trapeze method

def Beta_Func(span_a, alpha_a, beta_a):
    ris = 0.
    for i in range(len(span_a)- 1):
        mid_a = 0.5*(span_a[i + 1] + span_a[i])
        ris +=  (span_a[i + 1] - span_a[i])*(np.power(mid_a,(alpha_a - 1.))*np.power((1. - mid_a),(beta_a - 1.)))
   
    return ris

In [ ]:
def SpinModDistrib(a, alpha_a, beta_a, Beta_Val):
     
    return ((np.power(a, alpha_a - 1.)*np.power(1. - a, beta_a - 1.))/(Beta_Val))

while the spin orientations distribution is given by :

In [ ]:
def SpinOrientDistrib(cos_t1,cos_t2, zeta, sigma_1, sigma_2):
    prob = (1. - zeta)/(4) + ((2.*zeta)/(np.pi))*\
    (np.exp(-((np.power(1. - cos_t1,2.))/(2.*np.power(sigma_1,2.))))/(sigma_1*sc.erf(np.sqrt(2)/sigma_1)))\
    *(np.exp(-((np.power(1. - cos_t2,2.))/(2.*np.power(sigma_2,2.))))/(sigma_2*sc.erf(np.sqrt(2)/sigma_2)))
    return prob


We may finally define the distribution function for the number of events :

In [ ]:
def NDistrib(z,m1,m2,a_1,a_2,cos_t1,cos_t2):
    n = R_0*DeVC(z)*(T_obs/(1. + z)) \
    *MassDistr(m1,m2, m_min, m_max, alpha,beta_q) \
    *SpinModDistrib(a_1, alpha_a, beta_a, BetaVal)*SpinModDistrib(a_2, alpha_a, beta_a, BetaVal) \
    *SpinOrientDistrib(cos_t1, cos_t2, zeta, sigma_1, sigma_2)
    return n

The global variables of the simulation will be set to :

In [ ]:
# Mass Distribution parameters (values taken from the results of arxiv 1811.12940)

m_min = 5. # Solar Masses
m_max = 50. # Solar Masses
alpha = 1.6 # +-1.6 Big Error !
beta_q = 6.7 # +4.8 -5.9 Still Big Error !

#Spin Distribution parameter (values assumed considering the results of  arxiv 1811.12940)

Expected_a = 0.2 
Var_a = 1./40.
a_max = 1. 
alpha_a, beta_a = BetaSpinParameters(Expected_a, Var_a)
sigma_1 = 5.
sigma_2 = 5.
zeta = 1. # For a gaussian distribution of the spin alignment 

# Merger distribution parameters

T_obs = 3. # years estimated of observation
R_0 = 53.2 # +58.5 - 27.0 Gpc^-3 yr-1 Merger rate density assumed constant over the comoving volume
zmax = 0.5 # z value corrispondent to 2 gigaparsec

#General Constants 

c = 299792.46 # speed of light in Km/sec
H_0 = 67.8 # Hubble constant in Km/(s*MPc) 

if(alpha_a <=1 or beta_a <=1):
    print('Errore nella selezione dei valori di E[a] e Var[a]')

Let's plot the colormaps of the probability functions to check how they behave :

In [ ]:
# Colormap of the mass distribution function

Z = np.zeros((25,25))
ran_m1 = np.linspace(m_min,m_max,25)
ran_m2 = np.linspace(m_min,m_max,25)
X, Y = np.meshgrid(ran_m1, ran_m2)

for i in range(len(ran_m1)):
    for j in range(len(ran_m2)):
        Z[j][i] = MassDistr(X[j][i], Y[j][i], m_min, m_max, alpha, beta_q)
        
plt.contourf(X, Y, Z, 1000, cmap='jet')
plt.colorbar();
plt.xlabel(r'$m_1$', fontsize = 15)
plt.ylabel(r'$m_2*$', fontsize = 15)
plt.loglog()

In [ ]:
# Colormap of the mass distribution function

Z = np.zeros((25,25))
ran_a1 = np.linspace(0.,a_max,25)
ran_a2 = np.linspace(0.,a_max,25)
BetaVal = Beta_Func(ran_a1, alpha_a, beta_a)
X, Y = np.meshgrid(ran_a1, ran_a2)

for i in range(len(ran_a1)):
    for j in range(len(ran_a2)):
        Z[j][i] = SpinModDistrib(X[j][i], alpha_a, beta_a, BetaVal)*SpinModDistrib(Y[j][i], alpha_a, beta_a, BetaVal)
        
plt.contourf(X, Y, Z, 100, cmap='jet')
plt.colorbar();
plt.xlabel(r'$a_1$', fontsize = 15)
plt.ylabel(r'$a_2*$', fontsize = 15)
Z.max()


The simulation will be spanned over the following range of variables :

In [ ]:
ran_m1 = np.linspace(m_min,m_max,25)
ran_m2 = np.linspace(m_min,m_max,25)
ran_z = np.linspace(0.,zmax, 5)    # Don't put bins too small or the volume wouldn't be enough to generate events
ran_a_1 = np.linspace(0., a_max,5)
ran_a_2 = np.linspace(0., a_max,5)
BetaVal = Beta_Func(ran_a_1, alpha_a, beta_a)
ran_cos_t1 = np.linspace(-1.,1.,5)
ran_cos_t2 = np.linspace(-1.,1.,5)

and the result will be saved in the BHCat dataframe :

In [ ]:
BHCat = pd.DataFrame(columns=['EventName', 'Distance_GPc', 'Declination', 'RightAscension', 'Mass1', 'Mass2', 'Spin_a1', 'Spin_a2', 'cos_t1', 'cos_t2' ])
BHCat.set_index('EventName')

In [ ]:
# Example of the format of an added merging event
# BH= pd.DataFrame([['GW321312',1.1,0.6,0.8,22,18, 0.2,0., 0.4, -0.6],], columns=['EventName', 'Distance_GPc', 'Declination', 'RightAscension', 'Mass1', 'Mass2', 'Spin_a1', 'Spin_a2', 'cos_t1', 'cos_t2' ])

We may finally launch the pipeline to generate the merging events in the considered volume :

In [ ]:
Nev = 0
for im1 in range(len(ran_m1)-1):
    print('Percentage of completition :',(100.*ran_m1[im1]/m_max))
    for im2 in (range(im1)):
        for iz in range(len(ran_z) - 1):
            for ia_1 in range(len(ran_a_1)-1):
                for ia_2 in range(len(ran_a_2)-1):
                    for ict1 in range(len(ran_cos_t1)-1):
                        for ict2 in range(len(ran_cos_t2)-1):
                            # estimating the value of NDistrib in function of the values, the value will be interpolated with the trapeze method
                            nstep =  NDistrib(ran_z[iz],st.mean([ran_m1[im1],ran_m1[im1 + 1]]),st.mean([ran_m2[im2],ran_m2[im2 + 1]]),st.mean([ran_a_1[ia_1],ran_a_1[ia_1 + 1]])\
                                     ,st.mean([ran_a_2[ia_2],ran_a_2[ia_2 + 1]]),st.mean([ran_cos_t1[ict1],ran_cos_t1[ict1 + 1]]),st.mean([ran_cos_t2[ict2],ran_cos_t2[ict2 + 1]]))
                            # to obtain the real result of the integral, we now need to multiply the value for the delta of all the integration variables  
                            nstep *= Z_to_Gpc(ran_z[iz +1] - ran_z[iz])*(ran_m1[im1 + 1] - ran_m1[im1])*(ran_m2[im2 + 1] - ran_m2[im2])*(ran_a_1[ia_1 + 1] - ran_a_1[ia_1])*(ran_a_2[ia_2 + 1] - ran_a_2[ia_2])\
                                     *(ran_cos_t1[ict1 + 1] - ran_cos_t1[ict1])*(ran_cos_t2[ict2 + 1] - ran_cos_t2[ict2])
                            # The value need to be round up to an integer
                            nstep = round(nstep)
                            Nev += nstep
                            
print('Sono stati simulati ', Nev, 'diversi eventi')                            